In [1]:
# read all files in the folder: wiki

import os
import json
from tqdm import tqdm
import random

### Read the wikipedia dataset and collect metadata

In [2]:
info = []

def read_files():
    folder = 'wiki'
    for filename in tqdm(os.listdir(folder)):
        with open(os.path.join(folder, filename), 'r') as file:
            ff = json.loads(file.read())
            for i in ff:
                info.append({
                    "id": i['id'],
                    "length": len(i['text']),
                    "filename": filename
                })

read_files()


  0%|          | 0/242 [00:00<?, ?it/s]

100%|██████████| 242/242 [02:31<00:00,  1.59it/s]


### sort based on length of wikipedia text

In [3]:
info = sorted(info, key=lambda x: x.get('length', 0))

### Create and concatenate copies to make a large dataset with varied input sizes

In [ ]:
# 60, 30, 15, 5, 1 times each file
ind = [
-1,
-100,
-500,
-1000,
-8000,
-30000,
-60000,
-100000,
-150000,
-200000
]

def func(d):
    with open('wiki/' + d['filename'], 'r') as file:
        ff = json.loads(file.read())
        for i in ff:
            if i['id'] == d['id']:
                return i['text']

cnt = 0
for i in ind:
    a = func(info[i])
    with open('input_files/text/' + str(cnt) + '.txt', 'w') as file:
        file.write( (a) )
    cnt += 1

### Insert random sentences to ensure regex detection

In [34]:
aaa = [
    # Easy Patterns
    ["12", "45", "89", "00", "37"],  # \b[0-9]{2}\b
    ["cat", "The cat", "cat food", "cat.", "a cat!"],  # \bcat\b
    ["# Header", "# TODO", "# Note:", "#Start", "# End"],  # ^#
    [".txt", ".pdf", ".doc", ".jpg", ".png"],  # \.[a-z]{3}\b
    ["A", "B", "X", "M", "Z"],  # \b[A-Z]\b
    ["   text", "     start", "\tbegin", "  data", "\t  line"],  # ^[[:space:]]+
    ["abc", "xyz", "123", "def", "890"],  # \b[[:alnum:]]{3}\b
    ["1-2", "3-4", "5-6", "7-8", "9-0"],  # [0-9]-[0-9]
    ["apple", "eagle", "input", "ocean", "umbrella"],  # [aeiou]
    ["0", "1", "0", "1", "0"],  # \b[01]\b
    ["R", "G", "B", "RGB", "R G B"],  # \b[RGB]\b
    [", text", ",  space", ", data", ", next", ", end"],  # ,[[:space:]]
    ["hello", "world", "test", "code", "data"],  # \b[a-z]+\b
    ["1 ", "2 text", "3 data", "4 ", "5 end"],  # [0-9][[:space:]]
    ["N", "S", "E", "W", "NSEW"],  # \b[NSEW]\b
    ["_a", "_1", "_x", "_9", "_Z"],  # _[[:alnum:]]
    ["y", "n", "Y", "N", "y/n"],  # \b[yn]\b
    ["(1)", "(2)", "(3)", "(4)", "(5)"],  # \([0-9]\)
    ["AB", "CD", "XY", "PQ", "TZ"],  # \b[A-Z]{2}\b
    ["123", "456", "789", "000", "999"],  # \b[0-9]{3}\b
    
    # Medium Patterns
    ["12", "345", "6789", "00", "1234"],  # \b[0-9]{2,4}\b
    ["John", "Mary", "Thomas", "Alice", "Peter"],  # \b[A-Z][a-z]+\b
    ["1: ", "2, ", "42: ", "99, ", "100: "],  # ^[0-9]+[,:][[:space:]]
    ["user@domain", "test@example", "a@b", "name@site", "info@host"],  # \b[[:alnum:]]+@[[:alnum:]]+\b
    ["1.23", "45.67", "0.89", "12.3", "4.56"],  # \b[0-9]{1,3}\.[0-9]{1,3}\b
    ["log", "info", "error", "LOG", "ERROR"],  # \b(log|info|error)\b
    ["12:34", "00:00", "23:59", "01:30", "22:15"],  # \b[0-9]{2}:[0-9]{2}\b
    ["abc123", "test456", "code789", "data000", "info111"],  # [A-Za-z]+[0-9]+
    ["get", "post", "put", "GET", "POST"],  # \b(get|post|put)\b
    ["main()", "test()", "init()", "run()", "stop()"],  # \b[[:alnum:]]+\(\)
    ["\"hello\"", "\"test\"", "\"data\"", "\"\"", "\"123\""],  # "[^"]*"
    ["2024-01-01", "2023-12-31", "2025-06-15", "2024-02-29", "2023-07-22"],  # \b[0-9]{4}-[0-9]{2}-[0-9]{2}\b
    ["0x1F", "0xFF", "0x00", "0xAB", "0x12"],  # \b(0x[0-9A-F]+)\b
    ["true", "false", "null", "TRUE", "FALSE"],  # \b(true|false|null)\b
    ["// Comment", "//Note", "// TODO", "//Test", "// Fix"],  # ^[[:space:]]*//.*$
    ["snake_case_var", "user_name_field", "data_type_int", "first_last_name", "min_max_value"],  # \b([a-z]+_){2,}\b
    ["IBM", "NASA", "FBI", "CIA", "HTML"],  # \b[A-Z]{3,}\b
    ["5k", "10mb", "2g", "500kb", "1gb"],  # \b[0-9]+[[:space:]]*[kmgb]\b
    ["<!--test-->", "<!-- Note -->", "<!--TODO-->", "<!-- Fix -->", "<!--End-->"],  # <!--.*-->
    ["11111111", "00000000", "10101010", "11110000", "00001111"],  # \b([01]{8})\b

    # Hard Patterns
    ["Test1", "Password2", "Secret3", "Secure4", "Strong5"],  # ^(?=.*[A-Z])(?=.*[0-9]).*$
    ["192.168.1.1", "10.0.0.1", "172.16.0.1", "127.0.0.1", "8.8.8.8"],  # IPv4
    ["test@example.com", "user@domain.co", "info@site.org", "name@host.io", "admin@server.net"],  # Email validator
    ["1234 5678 9012 3456", "1234-5678-9012-3456", "1234567890123456", "0000 0000 0000 0000", "9999-9999-9999-9999"],  # Card number
    ["http://example.com", "https://test.org", "http://site.net", "https://domain.io", "http://server.com"],  # URL
    ["00:11:22:33:44:55", "AA:BB:CC:DD:EE:FF", "01-23-45-67-89-AB", "FF-EE-DD-CC-BB-AA", "12:34:56:78:90:AB"],  # MAC
    ["<div>test</div>", "<p>text</p>", "<span>data</span>", "<h1>title</h1>", "<a>link</a>"],  # HTML tag
    ["{data}", "[1,2,3]", "  {test}  ", " [a,b,c] ", "{\"key\":\"value\"}"],  # JSON
    ["key=value", "name=John", "age=25", "color=blue", "size=large"],  # Key-value
    ["Pass123!@#", "Secret4$%^", "Secure5&*(", "Strong6)_+", "Complex7<>?"],  # Password
    ["1", "42", "128", "200", "255"],  # Numbers 1-255
    ["(())", "[[]]", "{{}}", "([])", "{[()]}"],  # Balanced brackets
    ["test.user@domain.com", "name_123@server.org", "info.desk@company.net", "user-name@host.io", "admin.sys@site.com"],  # Extended email
    ["hat", "pet cat", "big dog", "red cup", "tea pot"],  # Three vowels
    ["SecurePass123!", "StrongPass456@", "ComplexPass789#", "SafePass000$", "GoodPass321%"],  # Complex password
    ["1", "42", "128", "200", "255"],  # Extended 0-255
    ["2024-01-01", "2023/12/31", "2025.06.15", "2024-02-29", "2023/07/22"],  # Date
    ["(test)", "[data]", "{code}", "(a+b)", "[1,2,3]"],  # Matched brackets
    ["abcd1234", "Test9876", "Code5432", "Pass1098", "User7654"],  # No triple chars
    ["TestPass123!@#", "SecureKey456$%^", "StrongPwd789&*(", "SafeCode321)_+", "ComplexPwd567<>?"]  # Advanced password
]
print()

In [ ]:
def insert_sentences_randomly(fp, examples):
    # Read the original content of the file
    with open(fp, 'r') as file:
        lines = file.read()

    # Ensure at least one sentence from each subgroup is selected
    sentences_to_insert = []
    for subgroup in examples:
        num_sentences = random.randint(1, len(subgroup))  # Random number of sentences to insert from this subgroup
        sentences_to_insert.extend(random.sample(subgroup, num_sentences))  # Randomly select sentences

    print(sentences_to_insert)
    print(len(sentences_to_insert))
    print()


    # Determine random positions in the original content
    insertion_points = sorted(random.sample(range(len(lines) + 1), len(sentences_to_insert)))
    # Insert the sentences at the determined positions
    for i, sentence in enumerate(sentences_to_insert):
        lines = lines[:insertion_points[i]] + sentence + '\n' + lines[insertion_points[i]:]
        # lines.insert(insertion_points[i], sentence + '\n')

    # Write the modified content back to the file
    with open(fp, 'w') as file:
        file.writelines(lines)

insert_sentences_randomly('input_files/text/0.txt', aaa)
insert_sentences_randomly('input_files/text/1.txt', aaa)
insert_sentences_randomly('input_files/text/2.txt', aaa)
insert_sentences_randomly('input_files/text/3.txt', aaa)
insert_sentences_randomly('input_files/text/4.txt', aaa)
insert_sentences_randomly('input_files/text/5.txt', aaa)
insert_sentences_randomly('input_files/text/6.txt', aaa)
insert_sentences_randomly('input_files/text/7.txt', aaa)
insert_sentences_randomly('input_files/text/8.txt', aaa)
insert_sentences_randomly('input_files/text/9.txt', aaa)

### Save the file

In [41]:
def func(i):
    with open('input_files/text/' + str(i)+".txt", 'r') as file:
        return file.read()

cnt = 0
for i in tqdm(range(10)):
    for j in [120, 60, 30, 15, 5, 1]:
        a = func(i)
        with open('input_files/text_new/' + str(cnt) + '.txt', 'w') as file:
            file.write( (a + " ") * j )
        cnt += 1

100%|██████████| 10/10 [00:01<00:00,  8.11it/s]
